<a href="https://colab.research.google.com/github/jalorenzo/SparkNotebookColab/blob/master/BDF_03_Working_with_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#00 - Configuration of Apache Spark on Collaboratory


###Installing Java, Spark, and Findspark


---


This code installs Apache Spark 2.4.4, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [ ]:
import os

os.environ["SPARK_VERSION"] = "spark-3.5.0"
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://apache.osuosl.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!echo $SPARK_VERSION-bin-hadoop3.tgz
!rm $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

### Set Environment Variables
Set the locations where Spark and Java are installed.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark/"
os.environ["DRIVE_DATA"] = "/content/gdrive/My Drive/Enseignement/2023-2024/ING3/HPDA/BigDataFrameworks/data/"

!rm /content/spark
!ln -s /content/$SPARK_VERSION-bin-hadoop3 /content/spark
!export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
!echo $SPARK_HOME
!env |grep  "DRIVE_DATA"

### Start a SparkSession
This will start a local Spark session.

In [ ]:
!python -V

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Example: shows the PySpark version
print("PySpark version {0}".format(sc.version))

# Example: parallelise an array and show the 2 first elements
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

In [ ]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')



---


# 03 - Working with DataFrames

## Introduction to DataFrames
We will see:

  - How to create a DataFrame
  - Basic operations on DataFrames
      - Show rows
      - Select columns
      - Rename, add and delete columns
      - Delete null values and duplicated rows
      - Replace values
  - Save DataFrames in different formats

## Creating DataFrames
A DataFrame can be created in different ways:

  - From a data sequence
  - From Row-type objects
  - From an RDD or a DataSet
  - Reading data from a file
      - Like in Hadoop, Spark supports different filesystems: local, HDFS, Amazon S3
          - By and large, it supports any data source that can be read with Hadoop
      - Spark can access different types of files: plain text, CSV, JSON, [Parquet](https://parquet.apache.org/), [ORC](https://orc.apache.org/), Sequence, etc
        -   It also supports compressed files
  - Accessing relational databases or noSQL databases
    -   MySQL, Postgres, etc. using JDBC/ODBC
    -  Hive, HBase, Cassandra, MongoDB, AWS Redshift, etc.
    
Some examples on how to create DataFrames below:

### From a sequence or a list of data

In [ ]:
from pyspark.sql.functions import col,expr
# Creating a DataFrame from a range and adding two columns
df = spark.range(1,7,2).toDF("n")
df.show()
df.withColumn("n1", col("n")+1).withColumn("n2", expr("2*n")).show()
# Note that in the call to 'expr' we can include SQL code

In [ ]:
# DataFrame from a list of tuples
l = [("Eric", 5.1, "Pass"),\
     ("John", 4.0, "Fail"),\
     ("Manuel", None, None)]
dfMarks = spark.createDataFrame(l, schema=["Name", "mark", "result"])
dfMarks.show()
dfMarks.printSchema()

### Creating DataFrames with a schema


When creating a DataFrame, it is a good idea to specify its schema:

  - The schema defines the names and data types of each column
  - It uses an object of type ``StructType`` to define the name and type of the columns
  - The data types used by Spark are defined in:
      - For PySpark: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types
      - For Scala: https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.types.package


In [ ]:
from pyspark.sql.types import StructField, StructType, FloatType, StringType
from pyspark.sql import Row
# Define the DataFrame schema
schemaMarks = StructType([
    StructField("Name", StringType(), False),
    StructField("mark", FloatType(), True),
    StructField("result", StringType(), True)
    ])

# Create the DataFrame from a list of Row objects
rows = [Row("Eric", 5.1, "Pass"),\
         Row("John", 4.0, "Fail"),\
         Row("Manuel", None, None)]

dfMarks = spark.createDataFrame(rows, schema=schemaMarks)
dfMarks.show()
dfMarks.printSchema()

### Creating DataFrames from a text file


Each file line is stored as a row

In [ ]:
# Mount first the Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

dfQuijote = spark.read.text(os.environ["DRIVE_DATA"] + "/quijote.txt")
dfQuijote.show(truncate=False)

### Creating DataFrames from a CSV file (revisited)

As an example, we are going to use a file with questions and replies from Stack Exchange (https://stackexchange.com/) in Italian.
It is a CVS file, with the following 13 fields:

  0. ``nComs`` - Number of comments of the question of the reply
  2. ``lastActivity`` - Date and hour of the last modification
  3. ``userId`` - Owner's ID
  4. ``body`` - Text of the question or reply
  5. ``score`` - Score of the question or reply based on positive and negative votes
  6. ``creationDate`` - Creation date and hour
  6. ``numViewed`` - Number of times viewed (null if the question has never been viewed)
  7. ``title`` - Question title (null if it is a reply)
  8. ``tags`` - Tags assigned to the question (null if there are no tags assigned)
  9. ``nAnswers`` - Number of replies related to the question (null if there are not any)
  10. ``acceptedAnswerId`` - The ID of the accepted answer (null if the question has no accepted answer)
  11. ``postType`` - Type of message: 1 question, 2 reply
  12. ``id`` - Unique message identifier

Fields are separated by the "~" symbol

#### a) Read the file and infer the schema

In [ ]:
dfSEInferred = spark.read.format("csv")\
                    .option("mode", "FAILFAST")\
                    .option("sep", "~")\
                    .option("inferSchema", "true")\
                    .option("header", "false")\
                    .option("nullValue", "null")\
                    .option("compression", "bzip2")\
                    .load(os.environ["DRIVE_DATA"] +"italianPosts.csv.bz2")

Some options:

1. ``mode``: specifies what to do when it finds corrupted entries
    - ``PERMISSIVE``: sets all fields to null when a corrupted entry is found (default value)
    - ``DROPMALFORMED``: deletes the rows with corrupted entries
    - ``FAILFAST``: returns an error when a corrupted entry is found
2. ``sep``:  field delimiter (by default ",")
3. ``inferSchema``: whether column types must be inferred (by default "false")
4. ``header``: if "true", the first line is taken as the header (by default "false")
5. ``nullValue``: character or string thar represents a NULL in the file  (by default "")
6. ``compression``: compression type (by default "none")
  
These options are similar for other types of files.

In [ ]:
# Show 5 rows
dfSEInferred.show(5)

In [ ]:
# Find out how the schema was inferred
dfSEInferred.schema

In [ ]:
# Another way of getting the same result
dfSEInferred.printSchema()

#### b) Read the file and specify the schema

In [ ]:
from pyspark.sql.types import *
# We first create a list with each column header
# Note: avoid spaces and non-ascii characters on column names
header = (["nComs", "lastActivity", "userId",
            "body", "score", "creationDate", "numViewed", "title",
            "tags", "nAnswers", "acceptedAnswerId", "postType", "id"])

# Define the schema for the elements of the table
# StructType -> Defines a schema for the DF from a list of StructFields
# StructField -> Defines the name and type of each column, and whether it is nullable or not (True field)
dfSE_Schema = StructType([
  StructField(header[0], IntegerType(), True),
  StructField(header[1], TimestampType(), True),
  StructField(header[2], LongType(), True),
  StructField(header[3], StringType(), True),
  StructField(header[4], IntegerType(), True),
  StructField(header[5], TimestampType(), True),
  StructField(header[6], IntegerType(), True),
  StructField(header[7], StringType(), True),
  StructField(header[8], StringType(), True),
  StructField(header[9], IntegerType(), True),
  StructField(header[10], LongType(), True),
  StructField(header[11], ByteType(), True),
  StructField(header[12], LongType(), True)
  ])

dfSE = spark.read.format("csv")\
                    .option("mode", "FAILFAST")\
                    .option("sep", "~")\
                    .option("inferSchema", "false")\
                    .option("header", "false")\
                    .option("nullValue", "null")\
                    .option("compression", "bzip2")\
                    .schema(dfSE_Schema)\
                    .load(os.environ["DRIVE_DATA"] +"italianPosts.csv.bz2")
dfSE.cache()

In [ ]:
dfSE.sort("id").show()

In [ ]:
dfSE.printSchema()

## Basic operations with DataFrames

### Show rows

In [ ]:
# show(n) shows the first n rows (by default, n=20)
dfSE.show(5)

In [ ]:
# Say that we do not want to truncate the long fields
dfSE.show(5, truncate=False)

In [ ]:
# take(n) returns the first n rows as a Python list of Row objects
list = dfSE.take(5)
print(list[1])
print("\n")
# collect() returns the DataFrame as a Python list of Row objects
# Warning: if the DataFrame is too large, it might collapse the Driver!
list2 = dfSE.collect()
print(list2[10])

In [ ]:
import os
# sample(withReplacement, fraction, seed=None) returns a new Dataframe with a fraction of the original rows
dfSESampled = dfSE.sample(False, 0.1, seed=None)
print("Original Number of rows = {0}; Number of sampled rows = {1}".format(dfSE.count(), dfSESampled.count()))

In [ ]:
# limit(n) limits the number of rows calculated to n
dfSE_10rows = dfSE.sample(False, 0.1, seed=None).limit(10)
print("Number of sampled rows = {0}".format(dfSE_10rows.count()))
dfSE_10rows.show()

### Execute an operation on each row
The method `foreach` applies a function to each row

- The DataFrame is not modified and no other DataFrames are created
- `foreach` is executed in the Workers

In [ ]:
def printid(f):
    print(f["id"])

# In theory, this code should print all values of the 'id' column.
# Due to the way the notebook manages tasks, it is not possible to see any output.
# Run it on a pyspark-shell to see the output.
dfSE_10rows.foreach(printid)


### Select columns

In [ ]:
# Creates a new DataFrame by selecting columns by name
dfIdBody = dfSE.select("id", "body")
dfIdBody.show(5)

print("The idBody object is of type {0}".format(type(dfIdBody)))

In [ ]:
# Another way of specifying the columns to select
dfIdBody2 = dfSE.select(dfSE.id, dfSE.body)
dfIdBody2.show(5)

In [ ]:
# It is also possible to specify objects of Column type...
from pyspark.sql.functions import col

colId = col("id")
colCreateDate = col("creationDate")
print("The colId object is of type {0}".format(type(colId)))
print("The colCreateDate object is of type {0}".format(type(colCreateDate)))

In [ ]:
# ... and create a DataFrame from Column objects, by renaming the columns
dfIdBodyDate = dfSE.select(colId,
                              colCreateDate.alias("Creation_date"),
                              dfSE.body.alias("Content"))
dfIdBodyDate.show(5)

#### Select columns by using expressions

To select columns using SQL expressions

In [ ]:
from pyspark.sql.functions import expr
# Same DataFrame as before but using expressions
dfIdDateBodyExpr = dfSE.select(
                           expr("id AS ID"),
                           expr('creationDate AS Creation_date'),
                           expr("body AS Content"))
dfIdDateBodyExpr.show(5)


In [ ]:
# We can use more complex expressions
dfSE.selectExpr("*", # Select all columns and set ValidReply to True for those with, at least, one reply.
                "(nAnswers IS NOT NULL) as ValidReply").show()

### Rename, add and delete columns


In [ ]:
# Rename the creationDate column
dfSE = dfSE.withColumnRenamed("creationDate", "Creation_date")
dfSE.cache()
dfSE.select("Creation_date",
            dfSE.numViewed.alias("Number_of_visits"),
            "score",
            "postType").show(truncate=False)

In [ ]:
# Add a new column 'ones' with all its values set to 1
from pyspark.sql.functions import lit
# lit transforms a literal in Python to Spark internal format
# (in this example, IntegerType)
dfSE = dfSE.withColumn("ones", lit(1))
dfSE.show(5)

In [ ]:
# Removes a column using drop
dfSE = dfSE.drop(col("ones"))
dfSE.columns

### Delete null and duplicated values

In [ ]:
# Remove all rows that have null on any of their columns
dfNoNulls = dfSE.dropna("any")
print("Initial number or rows: {0}; number of non null rows: {1}"
       .format(dfSE.count(), dfNoNulls.count()))


In [ ]:
# Remove rows that have null on all their columns
dfNeitherNull = dfSE.dropna("all")
print("Number of rows with all columns set to null: {0}"
       .format(dfSE.count() - dfNeitherNull.count()))

In [ ]:
# Remove duplicated rows
dfWithoutDuplicates = dfSE.dropDuplicates()
print("Number of duplicated rows: {0}"
       .format(dfSE.count() - dfWithoutDuplicates.count()))

In [ ]:
# Remove rows when a given column is duplicated
dfWithoutDuplicatedUser = dfSE.dropDuplicates(["userId"])
print("Number of unique users: {0}"
       .format(dfWithoutDuplicatedUser.count()))

In [ ]:
# Other examples
dfNoNullnumViewedAcceptedAnswerId = dfSE.dropna("any", subset=["numViewed", "acceptedAnswerId"])
print("Number of rows with numViewed AND acceptedAnswerId not null: {0}"
       .format(dfNoNullnumViewedAcceptedAnswerId.count()))

dfNoNullnumViewedAcceptedAnswerId = dfSE.dropna("all", subset=["numViewed", "acceptedAnswerId"])
print("Number of rows with numViewed OR acceptedAnswerId not null: {0}"
       .format(dfNoNullnumViewedAcceptedAnswerId.count()))

### Replacing values

In [ ]:
# Replace with '0' all null values in the numVistas and nAnswers fields
dfSE = dfSE.fillna(0, subset=["numViewed", "nAnswers"])
dfSE.show(5)

In [ ]:
# Replace the value 1170 with 3000 in columns "id" and "acceptedAnswerId"
dfSE.select("id", "acceptedAnswerId").show(10)
dfSE.replace(1170, 3000, subset=["id", "acceptedAnswerId"])\
    .select("id", "acceptedAnswerId")\
    .show(10)

## Saving DataFrames

As for reading, Spark can save DateFrames in multiple formats:

- CSV, JSON, Parquet, Hadoop...

It can write them as well on a database

In [ ]:
# Save the dfSE DataFrame in JSON format
#dfSE.write.format("json").mode("overwrite").save("/content/dfSE.json")
dfSE.write.json(os.environ["DRIVE_DATA"] + "dfSE.json",mode="overwrite")

#!mv /content/dfSE.json "$DRIVE_DATA"

In [ ]:
!ls -alh "$DRIVE_DATA"/dfSE.json

In [ ]:
# Save the DataFrame using Parquet
dfSE.write.format("parquet")\
    .mode("overwrite")\
    .save(os.environ["DRIVE_DATA"] + "dfSE.parquet")

In [ ]:
# Parquet uses by default the Snappy compressed format
!ls -alh "$DRIVE_DATA"/dfSE.parquet

It will create as many files as there are partitions in the DataFrame

In [ ]:
dfSE2 = dfSE.repartition(2)
# Save the DataFrame using Parquet, with gzip compression
dfSE2.write.format("parquet")\
     .mode("overwrite")\
     .option("compression", "gzip")\
     .save(os.environ["DRIVE_DATA"] + "/dfSE2.parquet")

In [ ]:
!ls -alh "$DRIVE_DATA"/dfSE2.parquet

### Partitioning

Spark can partition and save a file using the value of a given column

- A directory is created for each different value in the partitioning column
    - All data associated to that value are stored in that directory
- It simplifies the access to the values associated to a given key


In [ ]:
# Save our DataFrame partitioned by the userID field (using Parquet)
dfSE.write.format("parquet")\
    .mode("overwrite")\
    .partitionBy("userId")\
    .save(os.environ["DRIVE_DATA"] + "dfSE-partition.parquet")


In [ ]:
#!ls -lh "$DRIVE_DATA"dfSE-partition.parquet
!ls -lh "$DRIVE_DATA"dfSE-partition.parquet/userId=10
#rm -rf "$DRIVE_DATA"dfSE-partition.parquet



---

# Exercises


## Exercise 3.1: Word count

Count the number of words *per line* in the $DRIVE_DATA/quijote.txt file.

Repeat the exercise but this time counting the number of words *in the whole file*.

In [ ]:
from pyspark.sql import functions as F
# so that we can use the F.split() function.
